In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('papers_with_missing_affiliations.txt', delimiter='\t')

In [ ]:
latex_data = pd.read_csv('arXiv_data_with_latex.csv')

,Can a multi-tracer approach improve the constraints on the turnover scale?
0,First large scale spatial and velocity pattern...
1,A Stellar Magnesium to Silicon ratio in the at...
2,Photometric and spectroscopic variability of b...
3,Homogeneous measurements of proximity zone siz...
4,The COSMOS-Web Lens Survey (COWLS) III: foreca...
...,...
3330,CubeSat Orbit Insertion Maneuvering Using J2 P...
3331,New Bulgarian-Austrian project 'Joint observat...
3332,Year six photometric measurements of known Tra...
3333,"Alfvén wave propagation, reflection and trappi..."


In [8]:
list_countries = pd.read_csv('world_coords.csv')
list_countries = list_countries['country'].tolist()

In [9]:
list_countries

['Andorra',
 'United Arab Emirates',
 'Afghanistan',
 'Antigua and Barbuda',
 'Anguilla',
 'Albania',
 'Armenia',
 'Netherlands Antilles',
 'Angola',
 'Antarctica',
 'Argentina',
 'American Samoa',
 'Austria',
 'Australia',
 'Aruba',
 'Azerbaijan',
 'Bosnia and Herzegovina',
 'Barbados',
 'Bangladesh',
 'Belgium',
 'Burkina Faso',
 'Bulgaria',
 'Bahrain',
 'Burundi',
 'Benin',
 'Bermuda',
 'Brunei',
 'Bolivia',
 'Brazil',
 'Bahamas',
 'Bhutan',
 'Bouvet Island',
 'Botswana',
 'Belarus',
 'Belize',
 'Canada',
 'Cocos [Keeling] Islands',
 'Congo [DRC]',
 'Central African Republic',
 'Congo [Republic]',
 'Switzerland',
 "Côte d'Ivoire",
 'Cook Islands',
 'Chile',
 'Cameroon',
 'China',
 'Colombia',
 'Costa Rica',
 'Cuba',
 'Cape Verde',
 'Christmas Island',
 'Cyprus',
 'Czech Republic',
 'Germany',
 'Djibouti',
 'Denmark',
 'Dominica',
 'Dominican Republic',
 'Algeria',
 'Ecuador',
 'Estonia',
 'Egypt',
 'Western Sahara',
 'Eritrea',
 'Spain',
 'Ethiopia',
 'Finland',
 'Fiji',
 'Falkland 